## Imports

In [1]:
import xarray as xr
import pathlib
import numpy as np
import pandas as pd
import matplotlib as mpl
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xesmf as xe
import time
import src.utils
import copy

## specify filepath for data
DATA_FP = pathlib.Path(os.environ["DATA_FP"])

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI for presentation
mpl.rcParams["figure.dpi"] = 100

## Shared functions

In [2]:
def trim(
    data, lon_range=[130, 290], lat_range=[-5, 5], lon_name="TLONG", lat_name="TLAT"
):
    """select part of data in given longitude/latitude range"""

    ## helper function to check if 'x' is in 'x_range'
    isin_range = lambda x, x_range: (x_range[0] <= x) & (x <= x_range[1])

    ## get mask for data in given lon/lat range
    in_lon_range = isin_range(data[lon_name], lon_range)
    in_lat_range = isin_range(data[lat_name], lat_range)
    in_lonlat_range = in_lon_range & in_lat_range

    ## load to memory
    in_lonlat_range.load()

    ## Retain all points with at least one valid grid cell
    x_idx = in_lonlat_range.any("nlat")
    y_idx = in_lonlat_range.any("nlon")

    ## select given points
    return data.isel(nlon=x_idx, nlat=y_idx)

## Subsurface ocean data

In [ ]:
def get_ensemble_ids():
    """get files for given variable name"""

    ## path to cesm2 lens data
    cesm2_fp = pathlib.Path(
        "/glade/campaign/collections/rda/data/d651056/CESM2-LE/ocn/proc/tseries/month_1"
    )

    ## path to FSNS (arbitrary, just want the ids)
    data_fp = cesm2_fp / "WVEL"

    ## get list of ensemble ids
    ensemble_ids = []
    for f in data_fp.glob("*.nc"):
        ensemble_ids.append(str(f)[-53:-28])

    ## get unique values and sort
    ensemble_ids = sorted(list(set(ensemble_ids)))

    return ensemble_ids


def load_var(varname, ensemble_id):
    """Load variable for given ensemble ID"""

    ## get path to data
    cesm2_fp = pathlib.Path(
        "/glade/campaign/collections/rda/data/d651056/CESM2-LE/ocn/proc/tseries/month_1"
    )

    ## path to data
    data_fp = cesm2_fp / varname

    ## get z-coordinate name
    if varname == "TEMP":
        z_coord = "z_t"

    elif varname == "WVEL":
        z_coord = "z_w_top"

    else:
        print("Not a valid variable")
        return

    ## open data for ensemble member
    data = xr.open_mfdataset(
        data_fp.glob(f"*{ensemble_id}*.nc"),
        decode_timedelta=True,
        chunks={"time": 12, z_coord: 60, "nlat": 384, "nlon": 320},
        parallel=True,
    )[varname]

    ## trim to eq Pac
    data = trim(data, lat_range=[-1.5, 1.5], lon_range=[130, 290])

    ## subset longitude and get top 300 m
    data = data.isel({z_coord: slice(None, 27), "nlon": slice(None, None, 2)})

    ## average over latitudes
    data = data.assign_coords({"lon": data["TLONG"].mean("nlat")})
    data = data.mean("nlat")

    return data


def preprocess_ensemble(varname, temp_dir):
    """compute net heat flux for full ensemble. Save to temp directory"""

    ## get ensemble ids
    ensemble_ids = get_ensemble_ids()

    ## loop through members
    # idx 60 throwing "NetCDF: HDF error" for WVEL
    for i in tqdm.tqdm(ensemble_ids[61:]):
        # for i in tqdm.tqdm(ensemble_ids):

        ## save filepath
        save_fp = pathlib.Path(temp_dir, f"{varname}_{i}.nc")

        if save_fp.is_file():
            pass

        else:
            data = load_var(varname=varname, ensemble_id=i)
            data.to_netcdf(save_fp)

    return

#### Initialize cluster

In [ ]:
from dask.distributed import LocalCluster, Client

cluster = LocalCluster(n_workers=16)
client = Client(cluster)
client

In [ ]:
## compute wvel for each file
# preprocess_ensemble(varname="TEMP", temp_dir=pathlib.Path(DATA_FP, "cesm", "temp_temp_v2"))
preprocess_ensemble(
    varname="WVEL", temp_dir=pathlib.Path(DATA_FP, "cesm", "wvel_temp_v2")
)

### can't open one of the files; not sure why

In [ ]:
# ids = get_ensemble_ids()
# temp_dir = "wvel_temp"
# save_fp = pathlib.Path(DATA_FP, "cesm", "wvel_temp", f"WVEL_{ids[60]}.nc")

# ## load data
# d = load_var("WVEL", ids[60])

## Surface data ($u$, $v$)

In [3]:
def get_ensemble_ids():
    """get files for given variable name"""

    ## path to cesm2 lens data
    cesm2_fp = pathlib.Path(
        "/glade/campaign/collections/rda/data/d651056/CESM2-LE/ocn/proc/tseries/month_1"
    )

    ## path to FSNS (arbitrary, just want the ids)
    data_fp = cesm2_fp / "UVEL"

    ## get list of ensemble ids
    ensemble_ids = []
    for f in data_fp.glob("*.nc"):
        ensemble_ids.append(str(f)[-53:-28])

    ## get unique values and sort
    ensemble_ids = sorted(list(set(ensemble_ids)))

    return ensemble_ids

def load_var(varname, ensemble_id):
    """Load variable for given ensemble ID"""

    ## get path to data
    cesm2_fp = pathlib.Path(
        "/glade/campaign/collections/rda/data/d651056/CESM2-LE/ocn/proc/tseries/month_1"
    )

    ## path to data
    data_fp = cesm2_fp / varname

    ## open data for ensemble member
    data = xr.open_mfdataset(
        data_fp.glob(f"*{ensemble_id}*.nc"),
        decode_timedelta=True,
        chunks={"time": 60, "nlat": 384, "nlon": 320},
        parallel=True,
    )[varname]

    ## get top layer
    data = data.isel(z_t=0)

    ## trim to eq Pac
    data = trim(
        data,
        lat_range=[-15, 15],
        lon_range=[120, 300],
        lon_name="ULONG",
        lat_name="ULAT",
    )

    return data

def preprocess_ensemble(varname, temp_dir):
    """compute net heat flux for full ensemble. Save to temp directory"""

    ## get ensemble ids
    ensemble_ids = get_ensemble_ids()

    ## loop through members
    for i in tqdm.tqdm(ensemble_ids):

        ## save filepath
        save_fp = pathlib.Path(temp_dir, f"{varname}_{i}.nc")

        if save_fp.is_file():
            pass

        else:
            data = load_var(varname=varname, ensemble_id=i)
            data.to_netcdf(save_fp)

    return

### Initialize cluster

In [4]:
from dask.distributed import LocalCluster, Client

cluster = LocalCluster(n_workers=16)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kcarr/preprocess/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kcarr/preprocess/proxy/8787/status,Workers: 16
Total threads: 16,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45427,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kcarr/preprocess/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:34223,Total threads: 1
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kcarr/preprocess/proxy/36153/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:44917,


### Compute

In [ ]:
# ## test it works
# ids = get_ensemble_ids()
# d = load_var(varname="UVEL", ensemble_id=ids[0])

preprocess_ensemble(
    varname="UVEL", temp_dir=pathlib.Path(DATA_FP, "cesm", "uvel_temp")
)

 24%|██▍       | 24/100 [25:14<1:19:05, 62.45s/it]